In [1]:
#imports

import pandas as pd
import numpy as np
import os

from prepare import zillow_pipeline

In [2]:
df = zillow_pipeline()

In [3]:
df.head()

,bedrooms,bathrooms,area,value,year,county,state
0,4,3.5,3100,1023282,1998,Orange,CA
1,2,1.0,1465,464000,1967,Ventura,CA
2,3,2.0,1243,564778,1962,Orange,CA
3,4,3.0,2376,145143,1970,Los Angeles,CA
4,4,3.0,2962,773303,1950,Los Angeles,CA


In [4]:
df.shape

(52246, 7)